In [22]:
import xml.etree.ElementTree as ET  
import numpy as np
import time

In [23]:
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = '../../SBW-vectors-300-min5.txt'
cantidad = 10000000
start = time.time()
print ("Time to process word vectors with W2V with "+str(cantidad)+"words:")

wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

end = time.time()
print (end - start)

Time to process word vectors with W2V with 10000000words:
285.91940808296204


In [24]:
fastText_file_vec = '../../fasttext-sbwc.3.6.e20.vec'
cantidad = 10000000
start = time.time()
print ("Time to process word vectors with fastText with "+str(cantidad)+" words:")

fastTextVectors = KeyedVectors.load_word2vec_format(fastText_file_vec, limit=cantidad)

end = time.time()
print (end - start)

Time to process word vectors with fastText with 10000000 words:
259.8358108997345


In [25]:
glove_file_vec = '../../glove-sbwc.i25.vec'
cantidad = 10000000
start = time.time()
print ("Time to process word vectors with glove with "+str(cantidad)+" words:")

glovevectors = KeyedVectors.load_word2vec_format(glove_file_vec, limit=cantidad)

end = time.time()
print (end - start)

Time to process word vectors with glove with 10000000 words:
251.48377895355225


In [26]:
#Testing word2vec
try:
    print(glovevectors.get_vector('hiperbole'))
except:
    print('palabra no esta en dict')

palabra no esta en dict


In [27]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
stopWords = set(stopwords.words('spanish'))
spanishStemmer = SnowballStemmer("spanish")

#print("y" in stopWords)
#stemmer.stem("cordenada")

[nltk_data] Downloading package stopwords to /Users/Ruizo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
Tokenizer = RegexpTokenizer(r'\w+')
lines= 'Hola cómoo , estáas'
words = Tokenizer.tokenize(lines)

wordsStemmed = []
for word in words:
    word=spanishStemmer.stem(word)
    wordsStemmed.append(word)
print(words)
print(wordsStemmed)

['Hola', 'cómoo', 'estáas']
['hol', 'como', 'esta']


In [29]:
tree = ET.parse('general-tweets-test-tagged.xml')  
root = tree.getroot()
polarityNumberTest={}
polarities=['NONE', 'NEU', 'P', 'N+', 'P+', 'N']
tweetCount=0

start = time.time()
print ("Time to process data:")

for tweet in root:
    tweetId=tweet.find('tweetid').text
    
    tweetCount+=1
    polarityNumber=0
    polarity=tweet.find('sentiments').find('polarity').find('value').text
    position=polarities.index(polarity)
    polarityNumber+=position

    polarityNumberTest[tweetId]=polarityNumber

end = time.time()
print (end - start)
print('Cantidad de tweets \n'+ str(tweetCount))

Time to process data:
0.16602492332458496
Cantidad de tweets 
60798


In [30]:
tree = ET.parse('general-tweets-test.xml')  
root = tree.getroot()
tweetsTest=[]
tweetCount=0

start = time.time()
print ("Time to process data:")

for tweet in root:
    wordsStemmed = []
    polarityNumber=0
    tweetText=tweet.find('content').text
    tweetId=tweet.find('tweetid').text
    
    if tweetText and (tweetId in polarityNumberTest):
        tweetCount+=1
        words = Tokenizer.tokenize(tweetText)

        for word in words:
            word=spanishStemmer.stem(word)
            if word not in stopWords:
                wordsStemmed.append(word)

        newEntry=[wordsStemmed,polarityNumberTest[tweetId]]

        tweetsTest.append(newEntry)

end = time.time()
print (end - start)
print('Cantidad de tweets \n'+ str(tweetCount))

Time to process data:
50.09516787528992
Cantidad de tweets 
60798


In [31]:
tree = ET.parse('general-tweets-train-tagged.xml')  
root = tree.getroot()
tweets=[]
polarities=['NONE', 'NEU', 'P', 'N+', 'P+', 'N']
polarityTypes=['AGREEMENT', 'DISAGREEMENT']
tweetCount=0

start = time.time()
print ("Time to process data:")

for tweet in root:
    tweetCount+=1
    wordsStemmed = []
    polarityNumber=0
    tweetText=tweet.find('content').text
    
    if tweetText:
        words = Tokenizer.tokenize(tweetText)

        for word in words:
            word=spanishStemmer.stem(word)
            if word not in stopWords:
                wordsStemmed.append(word)

        polarity=tweet.find('sentiments').find('polarity').find('value').text
        position=polarities.index(polarity)
        polarityNumber+=position

        #polarityType=tweet.find('sentiments').find('polarity').find('type').text
        #position=polarityTypes.index(polarityType)
        #polarityNumber+=6*position

        newEntry=[wordsStemmed,polarityNumber]

        tweets.append(newEntry)

end = time.time()
print (end - start)
print ("Output as [[[ListOfWords],[polarityNumber]],[[ListOfWords],[polarityNumber]],...]")
print('Cantidad de tweets \n'+ str(tweetCount))

Time to process data:
5.887253999710083
Output as [[[ListOfWords],[polarityNumber]],[[ListOfWords],[polarityNumber]],...]
Cantidad de tweets 
7219


In [32]:
vectorizedPolarityInfo=[]

#Word2Vect
vectorizedTweetsInfoW2V=[]

#FastText
vectorizedTweetsInfoFT=[]

#Glove
vectorizedTweetsInfoG=[]

start = time.time()
print("Time to vectorize tweets:")

for tweet in tweets:
    vectorizedTweetsW2V=[]
    vectorizedTweetsFT=[]
    vectorizedTweetsG=[]
    for word in tweet[0]:
        try:
            vectorW2V = wordvectors.get_vector(word)
            vectorizedTweetsW2V.append(vectorW2V)
            
            vectorFT = fastTextVectors.get_vector(word)
            vectorizedTweetsFT.append(vectorFT)
            
            vectorG = glovevectors.get_vector(word)
            vectorizedTweetsG.append(vectorG)
        except:
            pass    
    vectorMeanW2V = np.mean(np.array(vectorizedTweetsW2V),axis=0)
    vectorMeanFT = np.mean(np.array(vectorizedTweetsFT),axis=0)
    vectorMeanG = np.mean(np.array(vectorizedTweetsG),axis=0)
    try:
        len(vectorMeanW2V)
        vectorizedTweetsInfoW2V.append(vectorMeanW2V.tolist())        
        vectorizedTweetsInfoFT.append(vectorMeanFT.tolist())        
        vectorizedTweetsInfoG.append(vectorMeanG.tolist())
        
        vectorizedPolarityInfo.append(tweet[1])
    except:
        pass

end = time.time()
print(end-start)

Time to vectorize tweets:


/Users/Ruizo/.pyenv/versions/3.4.6/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Ruizo/.pyenv/versions/3.4.6/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1.7773690223693848


In [33]:
vectorizedPolarityTestInfo=[]

#Word2Vect
vectorizedTweetsInfoTestW2V=[]

#FastText
vectorizedTweetsInfoTestFT=[]

#Glove
vectorizedTweetsInfoTestG=[]

start = time.time()
print("Time to vectorize tweets:")

for tweet in tweetsTest:
    vectorizedTweetsTestW2V=[]
    vectorizedTweetsTestFT=[]
    vectorizedTweetsTestG=[]
    for word in tweet[0]:
        try:
            vectorW2V = wordvectors.get_vector(word)
            vectorizedTweetsTestW2V.append(vectorW2V)
            
            vectorFT = fastTextVectors.get_vector(word)
            vectorizedTweetsTestFT.append(vectorFT)
            
            vectorG = glovevectors.get_vector(word)
            vectorizedTweetsTestG.append(vectorG)
        except:
            pass    
    vectorMeanTestW2V = np.mean(np.array(vectorizedTweetsTestW2V),axis=0)
    vectorMeanTestFT = np.mean(np.array(vectorizedTweetsTestFT),axis=0)
    vectorMeanTestG = np.mean(np.array(vectorizedTweetsTestG),axis=0)
    try:
        len(vectorMeanTestW2V)
        vectorizedTweetsInfoTestW2V.append(vectorMeanTestW2V.tolist())        
        vectorizedTweetsInfoTestFT.append(vectorMeanTestFT.tolist())        
        vectorizedTweetsInfoTestG.append(vectorMeanTestG.tolist())
        
        vectorizedPolarityTestInfo.append(tweet[1])
    except:
        pass

end = time.time()
print(end-start)

Time to vectorize tweets:


/Users/Ruizo/.pyenv/versions/3.4.6/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Ruizo/.pyenv/versions/3.4.6/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


12.829874992370605


In [34]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import datasets

#Entrenar
svm_W2V = svm.SVC(kernel='linear', C=1)
svm_FT = svm.SVC(kernel='linear', C=1)
svm_G = svm.SVC(kernel='linear', C=1)

In [35]:
start = time.time()
print ("Time to train model with W2V:")
svm_W2V.fit(vectorizedTweetsInfoW2V,vectorizedPolarityInfo)
end = time.time()
print(end-start)

start = time.time()
print ("Time to train model with FastText:")
svm_FT.fit(vectorizedTweetsInfoFT, vectorizedPolarityInfo)
end = time.time()
print(end-start)

start = time.time()
print ("Time to train model with Glove:")
svm_G.fit(vectorizedTweetsInfoG, vectorizedPolarityInfo)
end = time.time()
print(end-start)

Time to train model with W2V:
22.045307874679565
Time to train model with FastText:
20.58834409713745
Time to train model with Glove:
22.993741035461426


In [36]:
print("Training Set Accuracy W2V:",svm_W2V.score(vectorizedTweetsInfoW2V,vectorizedPolarityInfo))
print("Test Set Accuracy W2V:",svm_W2V.score(vectorizedTweetsInfoTestW2V,vectorizedPolarityTestInfo))

print("Training Set Accuracy FastText:",svm_FT.score(vectorizedTweetsInfoFT,vectorizedPolarityInfo))
print("Test Set Accuracy FastText:",svm_FT.score(vectorizedTweetsInfoTestFT,vectorizedPolarityTestInfo))

print("Training Set Accuracy Glove:",svm_G.score(vectorizedTweetsInfoG,vectorizedPolarityInfo))
print("Test Set Accuracy Glove:",svm_G.score(vectorizedTweetsInfoTestG,vectorizedPolarityTestInfo))

Training Set Accuracy W2V: 0.371364653244
Test Set Accuracy W2V: 0.470426455482
Training Set Accuracy FastText: 0.479166666667
Test Set Accuracy FastText: 0.451224361311
Training Set Accuracy Glove: 0.470078299776
Test Set Accuracy Glove: 0.425875608867


In [37]:
from sklearn.metrics import confusion_matrix

print ('\nConfussion matrix W2V:\n',confusion_matrix(vectorizedPolarityTestInfo, svm_W2V.predict(vectorizedTweetsInfoTestW2V)))
print ('\nConfussion matrix FastText:\n',confusion_matrix(vectorizedPolarityTestInfo, svm_FT.predict(vectorizedTweetsInfoTestFT)))
print ('\nConfussion matrix Glove:\n',confusion_matrix(vectorizedPolarityTestInfo, svm_G.predict(vectorizedTweetsInfoTestG)))



Confussion matrix W2V:
 [[ 9459     0   229    15  4966  6450]
 [  118     0    11     1   379   796]
 [  372     0    36     0   631   446]
 [  673     0    50    21   784  3026]
 [ 4536     0   155     5 11910  4023]
 [ 2223     0   107    11  1957  6968]]

Confussion matrix FastText:
 [[ 9326   657  2272  1063  3709  4092]
 [  100    94   177   142   260   532]
 [  246    44   643    37   296   219]
 [  482   130   327  1684   438  1493]
 [ 4133   540  2888   566 10187  2315]
 [ 1960   490  1135  1179  1201  5301]]

Confussion matrix Glove:
 [[8919  823 2661 1367 3341 4008]
 [ 110  136  183  162  241  473]
 [ 220   69  653   53  273  217]
 [ 538  169  431 1537  482 1397]
 [4146  757 3032  899 9398 2397]
 [1925  559 1201 1263 1256 5062]]
